# Dependencies

In [1]:
!pip install backtrader
!pip install yfinance
!pip install PyQt5

In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount = True)

Mounted at /content/drive


In [3]:
import backtrader as bt
import matplotlib.pyplot as plt
import yfinance as yf

import backtrader.plot
import matplotlib

# MACD Strategy

In [4]:
class BitmexComissionInfo(bt.CommissionInfo):
    params = (
        ("commission", 0.00075),
        ("mult", 1.0),
        ("margin", None),
        ("commtype", None),
        ("stocklike", False),
        ("percabs", False),
        ("interest", 0.0),
        ("interest_long", False),
        ("leverage", 1.0),
        ("automargin", False),
    )
def getsize(self, price, cash):
        """Returns fractional size for cash operation @price"""
        return self.p.leverage * (cash / price)

In [5]:
class MACD(bt.Strategy):
    params = (
        ("macd1", 12),
        ("macd2", 26),
        ("macdsig", 9),
        # Percentage of portfolio for a trade. Something is left for the fees
        # otherwise orders would be rejected
        ("portfolio_frac", 0.98),
    )

    def __init__(self):
        self.val_start = self.broker.get_cash()  # keep the starting cash
        self.size = None
        self.order = None

        self.macd = bt.ind.MACD(
            self.data,
            period_me1=self.p.macd1,
            period_me2=self.p.macd2,
            period_signal=self.p.macdsig,
        )
        # Cross of macd and macd signal
        self.mcross = bt.ind.CrossOver(self.macd.macd, self.macd.signal)
    
    def next(self):
        if self.order:
            return  # pending order execution. Waiting in orderbook

        print(
            f"DateTime {self.datas[0].datetime.datetime(0)}, "
            f"Price {self.data[0]:.2f}, Mcross {self.mcross[0]}, "
            f"Position {self.position.upopened}"
        )

        if not self.position:  # not in the market
            if self.mcross[0] > 0.0:
                print("Starting buy order")
                self.size = (
                    self.broker.get_cash() / self.datas[0].close * self.p.portfolio_frac
                )
                self.order = self.buy(size=self.size)
        else:  # in the market
            if self.mcross[0] < 0.0:
                print("Starting sell order")
                self.order = self.sell(size=self.size)
                
    def notify_order(self, order):
        """Execute when buy or sell is triggered
        Notify if order was accepted or rejected
        """
        if order.alive():
            print("Order is alive")
            # submitted, accepted, partial, created
            # Returns if the order is in a status in which it can still be executed
            return

        order_side = "Buy" if order.isbuy() else "Sell"
        if order.status == order.Completed:
            print(
                (
                    f"{order_side} Order Completed -  Size: {order.executed.size} "
                    f"@Price: {order.executed.price} "
                    f"Value: {order.executed.value:.2f} "
                    f"Comm: {order.executed.comm:.6f} "
                )
            )
        elif order.status in {order.Canceled, order.Margin, order.Rejected}:
            print(f"{order_side} Order Canceled/Margin/Rejected")
        self.order = None  # indicate no order pending

    def notify_trade(self, trade):
        """Execute after each trade
        Calcuate Gross and Net Profit/loss"""
        if not trade.isclosed:
            return
        print(f"Operational profit, Gross: {trade.pnl:.2f}, Net: {trade.pnlcomm:.2f}")

    def stop(self):
        """ Calculate the actual returns """
        self.roi = (self.broker.get_value() / self.val_start) - 1.0
        val_end = self.broker.get_value()
        print(
            f"ROI: {100.0 * self.roi:.2f}%%, Start cash {self.val_start:.2f}, "
            f"End cash: {val_end:.2f}"
        )

# Test Strategy - Backtrader

In [6]:
class TestStrategy(bt.Strategy):

    def log(self, txt, dt=None):
        ''' Logging function fot this strategy'''
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s' % (dt.isoformat(), txt))

    def __init__(self):
        # Keep a reference to the "close" line in the data[0] dataseries
        self.dataclose = self.datas[0].close

        # To keep track of pending orders and buy price/commission
        self.order = None
        self.buyprice = None
        self.buycomm = None

    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # Buy/Sell order submitted/accepted to/by broker - Nothing to do
            return

        # Check if an order has been completed
        # Attention: broker could reject order if not enough cash
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log(
                    'BUY EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                    (order.executed.price,
                     order.executed.value,
                     order.executed.comm))

                self.buyprice = order.executed.price
                self.buycomm = order.executed.comm
            else:  # Sell
                self.log('SELL EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                         (order.executed.price,
                          order.executed.value,
                          order.executed.comm))

            self.bar_executed = len(self)

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')

        self.order = None

    def notify_trade(self, trade):
        if not trade.isclosed:
            return

        self.log('OPERATION PROFIT, GROSS %.2f, NET %.2f' %
                 (trade.pnl, trade.pnlcomm))

    def next(self):
        # Simply log the closing price of the series from the reference
        self.log('Close, %.2f' % self.dataclose[0])

        # Check if an order is pending ... if yes, we cannot send a 2nd one
        if self.order:
            return

        # Check if we are in the market
        if not self.position:

            # Not yet ... we MIGHT BUY if ...
            if self.dataclose[0] < self.dataclose[-1]:
                    # current close less than previous close

                    if self.dataclose[-1] < self.dataclose[-2]:
                        # previous close less than the previous close

                        # BUY, BUY, BUY!!! (with default parameters)
                        self.log('BUY CREATE, %.2f' % self.dataclose[0])

                        # Keep track of the created order to avoid a 2nd order
                        self.order = self.buy()

        else:

            # Already in the market ... we might sell
            if len(self) >= (self.bar_executed + 5):
                # SELL, SELL, SELL!!! (with all possible default parameters)
                self.log('SELL CREATE, %.2f' % self.dataclose[0])

                # Keep track of the created order to avoid a 2nd order
                self.order = self.sell()

# Buy and Hold Strategy

In [7]:
class BuyAndHold_Buy(bt.Strategy):    
    def start(self):
        # set the starting cash
        self.val_start = self.broker.get_cash()     
    def nextstart(self):
        # Buy stocks with all the available cash
        size = int(self.val_start / self.data)
        self.buy(size=size)    
    def stop(self):
        # calculate the actual returns
        self.roi = (self.broker.get_value() / self.val_start) - 1.0
        print("ROI: %.2f, Cash: %.2f" % (100.0 * self.roi, self.broker.get_value()))

# Results

## Binance Dataset

In [8]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/Binance-BC-EDA.csv')
df = df.drop(df.columns[6:14], axis = 1)
df.to_csv('Binance-reduced.csv', index=False)

In [9]:
df3 = pd.read_csv('/content/Binance-reduced.csv')
df3

,Date,Open,High,Low,Close,Volume
0,2017-08-17,4261.48,4485.39,4200.74,4285.08,795.150377
1,2017-08-18,4285.08,4371.52,3938.77,4108.37,1199.888264
2,2017-08-19,4108.37,4184.69,3850.00,4139.98,381.309763
3,2017-08-20,4120.98,4211.08,4032.62,4086.29,467.083022
4,2017-08-21,4069.13,4119.62,3911.79,4016.00,691.743060
...,...,...,...,...,...,...
1593,2021-12-27,50775.48,52088.00,50449.00,50701.44,28779.582120
1594,2021-12-28,50701.44,50704.05,47313.01,47543.74,45853.339240
1595,2021-12-29,47543.74,48139.08,46096.99,46464.66,39498.870000
1596,2021-12-30,46464.66,47900.00,45900.00,47120.87,30352.295690


In [10]:
df2 = pd.read_csv('/content/drive/MyDrive/file1.csv')
df2

,Date,Open,High,Low,Close,Adj Close,Volume,Return,Day of Week
0,2014-09-17,465.864014,468.174011,452.421997,457.334015,457.334015,2.105680e+07,NaN,Wed
1,2014-09-18,456.859985,456.859985,413.104004,424.440002,424.440002,3.448320e+07,-0.071926,Thu
2,2014-09-19,424.102997,427.834991,384.532013,394.795990,394.795990,3.791970e+07,-0.069843,Fri
3,2014-09-20,394.673004,423.295990,389.882996,408.903992,408.903992,3.686360e+07,0.035735,Sat
4,2014-09-21,408.084991,412.425995,393.181000,398.821014,398.821014,2.658010e+07,-0.024659,Sun
...,...,...,...,...,...,...,...,...,...
2580,2021-10-14,57372.832031,58478.734375,56957.074219,57321.523438,57321.523438,3.661579e+10,-0.001386,Thu
2581,2021-10-15,57345.902344,62757.128906,56868.144531,61593.949219,61593.949219,5.178008e+10,0.074534,Fri
2582,2021-10-16,61609.527344,62274.476563,60206.121094,60892.179688,60892.179688,3.425096e+10,-0.011393,Sat
2583,2021-10-17,60887.652344,61645.523438,59164.468750,61553.617188,61553.617188,2.903237e+10,0.010862,Sun


### Test Strategy

In [11]:
data = bt.feeds.YahooFinanceCSVData(dataname='/content/drive/MyDrive/Binance-BC-EDA.csv',reverse=False)

cerebro = bt.Cerebro()

cerebro.adddata(data)

cerebro.addstrategy(TestStrategy)

#cerebro.addstrategy(BuyAndHold_Buy)

cerebro.broker.addcommissioninfo(BitmexComissionInfo())

cerebro.broker.setcommission(commission=0.001)

cerebro.broker.setcash(10000.0)

print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

cerebro.run()

print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())

Starting Portfolio Value: 10000.00
2017-08-17, Close, 795.15
2017-08-17, BUY CREATE, 795.15
2017-08-18, BUY EXECUTED, Price: 1251.50, Cost: 1251.50, Comm 1.25
2017-08-18, Close, 1199.89
2017-08-19, Close, 381.31
2017-08-20, Close, 467.08
2017-08-21, Close, 691.74
2017-08-22, Close, 966.68
2017-08-23, Close, 1001.14
2017-08-23, SELL CREATE, 1001.14
2017-08-24, SELL EXECUTED, Price: 756.58, Cost: 1251.50, Comm 0.76
2017-08-24, OPERATION PROFIT, GROSS -494.92, NET -496.93
2017-08-24, Close, 787.42
2017-08-25, Close, 573.61
2017-08-25, BUY CREATE, 573.61
2017-08-26, BUY EXECUTED, Price: 225.12, Cost: 225.12, Comm 0.23
2017-08-26, Close, 228.11
2017-08-27, Close, 350.69
2017-08-28, Close, 603.84
2017-08-29, Close, 603.55
2017-08-30, Close, 808.47
2017-08-31, Close, 556.96
2017-08-31, SELL CREATE, 556.96
2017-09-01, SELL EXECUTED, Price: 543.85, Cost: 225.12, Comm 0.54
2017-09-01, OPERATION PROFIT, GROSS 318.73, NET 317.96
2017-09-01, Close, 560.67
2017-09-02, Close, 929.15
2017-09-03, Close

### Buy and Hold Strategy

In [12]:
from datetime import datetime

data = bt.feeds.YahooFinanceCSVData(dataname='/content/Binance-reduced.csv', reverse=False)
#data = bt.feeds.PandasData(dataname=df,timeframe=1,openinterest=None)

cerebro = bt.Cerebro()

cerebro.adddata(data)

cerebro.addstrategy(BuyAndHold_Buy)

cerebro.broker.setcash(10000.0)

print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

cerebro.run()

print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())

Starting Portfolio Value: 10000.00
ROI: 0.00, Cash: 10000.00
Final Portfolio Value: 10000.00


### MACD Strategy

In [13]:
cerebro = bt.Cerebro()

cerebro.broker.set_cash(10000)

data = bt.feeds.YahooFinanceCSVData(dataname='/content/drive/MyDrive/Binance-BC-EDA.csv',reverse=False)

cerebro.resampledata(data, timeframe=bt.TimeFrame.Minutes, compression=60)

cerebro.addstrategy(MACD)

cerebro.broker.addcommissioninfo(BitmexComissionInfo())

# Add TimeReturn Analyzers to benchmark data
cerebro.addanalyzer(
    bt.analyzers.TimeReturn, _name="alltime_roi", timeframe=bt.TimeFrame.NoTimeFrame
)

cerebro.addanalyzer(
    bt.analyzers.TimeReturn,
    data=data,
    _name="benchmark",
    timeframe=bt.TimeFrame.NoTimeFrame,
)

results = cerebro.run()
st0 = results[0]

for alyzer in st0.analyzers:
    alyzer.print()

DateTime 2017-09-20 23:59:59.999989, Price 720.94, Mcross 0.0, Position 0
DateTime 2017-09-21 23:59:59.999989, Price 1001.65, Mcross 0.0, Position 0
DateTime 2017-09-22 23:59:59.999989, Price 838.97, Mcross 0.0, Position 0
DateTime 2017-09-23 23:59:59.999989, Price 752.79, Mcross 0.0, Position 0
DateTime 2017-09-24 23:59:59.999989, Price 661.64, Mcross 0.0, Position 0
DateTime 2017-09-25 23:59:59.999989, Price 727.99, Mcross 0.0, Position 0
DateTime 2017-09-26 23:59:59.999989, Price 526.73, Mcross 0.0, Position 0
DateTime 2017-09-27 23:59:59.999989, Price 628.17, Mcross 0.0, Position 0
DateTime 2017-09-28 23:59:59.999989, Price 849.79, Mcross 0.0, Position 0
DateTime 2017-09-29 23:59:59.999989, Price 1602.31, Mcross 1.0, Position 0
Starting buy order
Order is alive
Order is alive
Buy Order Completed -  Size: 6.116169779880298 @Price: 686.87 Value: 4201.01 Comm: 0.031508 
DateTime 2017-09-30 23:59:59.999989, Price 720.35, Mcross 0.0, Position 6.116169779880298
DateTime 2017-10-01 23:59:

## Yahoo Dataset

### Test Strategy

In [14]:
data = bt.feeds.YahooFinanceCSVData(dataname='/content/drive/MyDrive/file1.csv',reverse=False)

cerebro = bt.Cerebro()

cerebro.adddata(data)

cerebro.addstrategy(TestStrategy)

#cerebro.addstrategy(BuyAndHold_Buy)

cerebro.broker.addcommissioninfo(BitmexComissionInfo())

cerebro.broker.setcommission(commission=0.001)

cerebro.broker.setcash(10000.0)

print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

cerebro.run()

print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())

Starting Portfolio Value: 10000.00
2014-09-17, Close, 457.33
2014-09-18, Close, 424.44
2014-09-18, BUY CREATE, 424.44
2014-09-19, BUY EXECUTED, Price: 424.10, Cost: 424.10, Comm 0.42
2014-09-19, Close, 394.80
2014-09-20, Close, 408.90
2014-09-21, Close, 398.82
2014-09-22, Close, 402.15
2014-09-23, Close, 435.79
2014-09-24, Close, 423.20
2014-09-24, SELL CREATE, 423.20
2014-09-25, SELL EXECUTED, Price: 423.16, Cost: 424.10, Comm 0.42
2014-09-25, OPERATION PROFIT, GROSS -0.94, NET -1.79
2014-09-25, Close, 411.57
2014-09-25, BUY CREATE, 411.57
2014-09-26, BUY EXECUTED, Price: 411.43, Cost: 411.43, Comm 0.41
2014-09-26, Close, 404.42
2014-09-27, Close, 399.52
2014-09-28, Close, 377.18
2014-09-29, Close, 375.47
2014-09-30, Close, 386.94
2014-10-01, Close, 383.61
2014-10-01, SELL CREATE, 383.61
2014-10-02, SELL EXECUTED, Price: 383.99, Cost: 411.43, Comm 0.38
2014-10-02, OPERATION PROFIT, GROSS -27.44, NET -28.24
2014-10-02, Close, 375.07
2014-10-02, BUY CREATE, 375.07
2014-10-03, BUY EXECUT

### Buy and Hold

In [17]:
data = bt.feeds.YahooFinanceCSVData(dataname='/content/drive/MyDrive/file1.csv',reverse=False)

cerebro = bt.Cerebro()

cerebro.adddata(data)

cerebro.addstrategy(BuyAndHold_Buy)

cerebro.broker.setcash(10000.0)

print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

cerebro.run()

print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())

Starting Portfolio Value: 10000.00
ROI: 12929.54, Cash: 1302953.62
Final Portfolio Value: 1302953.62


### MACD

In [16]:
cerebro = bt.Cerebro()

cerebro.broker.set_cash(10000)

data = bt.feeds.YahooFinanceCSVData(dataname='/content/drive/MyDrive/file1.csv',reverse=False)

cerebro.resampledata(data, timeframe=bt.TimeFrame.Minutes, compression=60)

cerebro.addstrategy(MACD)

cerebro.broker.addcommissioninfo(BitmexComissionInfo())

# Add TimeReturn Analyzers to benchmark data
cerebro.addanalyzer(
    bt.analyzers.TimeReturn, _name="alltime_roi", timeframe=bt.TimeFrame.NoTimeFrame
)

cerebro.addanalyzer(
    bt.analyzers.TimeReturn,
    data=data,
    _name="benchmark",
    timeframe=bt.TimeFrame.NoTimeFrame,
)

results = cerebro.run()
st0 = results[0]

for alyzer in st0.analyzers:
    alyzer.print()

DateTime 2014-10-21 23:59:59.999989, Price 386.48, Mcross 0.0, Position 0
DateTime 2014-10-22 23:59:59.999989, Price 383.16, Mcross 0.0, Position 0
DateTime 2014-10-23 23:59:59.999989, Price 358.42, Mcross 0.0, Position 0
DateTime 2014-10-24 23:59:59.999989, Price 358.35, Mcross 0.0, Position 0
DateTime 2014-10-25 23:59:59.999989, Price 347.27, Mcross -1.0, Position 0
DateTime 2014-10-26 23:59:59.999989, Price 354.70, Mcross 0.0, Position 0
DateTime 2014-10-27 23:59:59.999989, Price 352.99, Mcross 0.0, Position 0
DateTime 2014-10-28 23:59:59.999989, Price 357.62, Mcross 0.0, Position 0
DateTime 2014-10-29 23:59:59.999989, Price 335.59, Mcross 0.0, Position 0
DateTime 2014-10-30 23:59:59.999989, Price 345.30, Mcross 0.0, Position 0
DateTime 2014-10-31 23:59:59.999989, Price 338.32, Mcross 0.0, Position 0
DateTime 2014-11-01 23:59:59.999989, Price 325.75, Mcross 0.0, Position 0
DateTime 2014-11-02 23:59:59.999989, Price 325.89, Mcross 0.0, Position 0
DateTime 2014-11-03 23:59:59.999989, 